In [1]:
import json

with open('pa-data-common.json', 'r') as f:
    data = json.load(f)
    
import openpyxl
from os import listdir
from os.path import isfile, join
mypath = '../vpscrape/getvirons/xlfiles/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

from plot_util import *

def letters(s):
    valids = []
    for character in s:
        if character.isalpha():
            valids.append(character)
    return ''.join(valids)

In [2]:
data['close_aas'] = {}

In [3]:
for f in onlyfiles:
    if 'full' not in f: continue
    xldir = mypath + f
    wb = openpyxl.load_workbook(xldir)
    v = f.split('_', 1)[-1].split('.')[0]
    resdict = []
    for name in wb.sheetnames:
            sh = wb[name]
            vitems = []
            for row in sh.iter_rows():
                    vitems.append([cell.value for cell in row])
            aas = []
            cnt = 0
            for i in range(len(vitems[0])):
                for j in range(2, len(vitems)):
                    if vitems[j][i] is not None and '.' in str(vitems[j][i]):
                        d = vitems[j][i]
                        if d < 5:
                            aas.append(letters(vitems[j][i+2]))
                            if vitems[j][i+3] != 'N/A': aas.append(letters(vitems[j][i+3]))
            resdict.extend(aas)
    cres = countlist(resdict)
    data['close_aas'][v] = cres
    data['data'][v]['common_gauge_aa'] = max(cres, key=cres.get)

In [11]:
del data['closest_aas']

In [12]:
del data['gauge_points']

In [13]:
data.keys()

dict_keys(['data', 'filters', 'properties', 'aas', 'close_aas'])

In [14]:
#was pa-data-close.json
with open('pa-data-final.json', 'w') as f:
    json_str = json.dumps(data, indent=4)
    f.write(json_str)